In [1]:
import pandas as pd
import numpy as np

'''
Load Data
'''
IDIR = 'G://bigdata//badou//00-data//'
priors = pd.read_csv(IDIR + 'order_products__prior.csv', dtype={
    'order_id': np.int32,
    'product_id': np.int16,
    'add_to_cart_order': np.int16,
    'reordered': np.int8})
train = pd.read_csv(IDIR + 'order_products__train.csv', dtype={
    'order_id': np.int32,
    'product_id': np.int16,
    'add_to_cart_order': np.int16,
    'reordered': np.int8})
orders = pd.read_csv(IDIR + 'orders.csv', dtype={
    'order_id': np.int32,
    'user_id': np.int32,
    'eval_set': 'object',
    'order_number': np.int16,
    'order_dow': np.int8,
    'order_hour_of_day': np.int8,
    'days_since_prior_order': np.float32})
products = pd.read_csv(IDIR + 'products.csv', dtype={
    'product_id': np.int16,
    'order_id': np.int32,
    'aisle_id': np.int8,
    'department_id': np.int8}, usecols=['product_id', 'aisle_id', 'department_id'])

In [2]:
print('priors {}: {}'.format(priors.shape, ', '.join(priors.columns)))
print('orders {}: {}'.format(orders.shape, ', '.join(orders.columns)))
print('train {}: {}'.format(train.shape, ', '.join(train.columns)))

priors (32434489, 4): order_id, product_id, add_to_cart_order, reordered
orders (3421083, 7): order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order
train (1384617, 4): order_id, product_id, add_to_cart_order, reordered


In [7]:
# 1. product feature
print('computing product f')
prods = pd.DataFrame()
# 产品销量 int32
prods['orders'] = priors.groupby('product_id').size().astype(np.int32)

computing product f


In [4]:
priors.product_id.head(10)

0   -32416
1    28985
2     9327
3   -19618
4    30035
5    17794
6   -25395
7     1819
8   -21868
9   -31782
Name: product_id, dtype: int16

In [8]:
prods.head(10)

,orders
product_id,
-32768,502
-32767,7
-32766,14
-32765,485
-32764,110
-32763,587
-32762,5
-32761,263
-32760,232


In [9]:
priors.product_id

0          -32416
1           28985
2            9327
3          -19618
4           30035
            ...  
32434484   -25858
32434485    11352
32434486     4600
32434487    24852
32434488     5020
Name: product_id, Length: 32434489, dtype: int16

In [10]:
priors.head(
)

,order_id,product_id,add_to_cart_order,reordered
0,2,-32416,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,-19618,4,1
4,2,30035,5,0


In [11]:
priors2 = pd.read_csv(IDIR + 'order_products__prior.csv', dtype={
    'order_id': str,
    'product_id': str,
    'add_to_cart_order': np.int16,
    'reordered': np.int8})

In [12]:
priors2.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [13]:
prods['reorders'] = priors['reordered'].groupby(priors.product_id).sum().astype(np.float32)

In [14]:
prods.head()

,orders,reorders
product_id,,
-32768,502,208.0
-32767,7,2.0
-32766,14,3.0
-32765,485,335.0
-32764,110,54.0


In [15]:
prods['reorder_rate'] = (prods.reorders / prods.orders).astype(np.float32)

In [16]:
products = products.join(prods, on='product_id')

In [17]:
products.head()

,product_id,aisle_id,department_id,orders,reorders,reorder_rate
0,1,61,19,1852.0,1136.0,0.613391
1,2,104,13,90.0,12.0,0.133333
2,3,94,7,277.0,203.0,0.732852
3,4,38,1,329.0,147.0,0.446809
4,5,5,13,15.0,9.0,0.600000


In [18]:
products.set_index('product_id', drop=False, inplace=True)

In [19]:
products.head()

,product_id,aisle_id,department_id,orders,reorders,reorder_rate
product_id,,,,,,
1,1,61,19,1852.0,1136.0,0.613391
2,2,104,13,90.0,12.0,0.133333
3,3,94,7,277.0,203.0,0.732852
4,4,38,1,329.0,147.0,0.446809
5,5,5,13,15.0,9.0,0.600000


In [20]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [21]:
orders.set_index('order_id', inplace=True, drop=False)

In [22]:
priors.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,-32416,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,-19618,4,1
4,2,30035,5,0


In [23]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,,
2539329,2539329,1,prior,1,2,8,NaN
2398795,2398795,1,prior,2,3,7,15.0
473747,473747,1,prior,3,3,12,21.0
2254736,2254736,1,prior,4,4,7,29.0
431534,431534,1,prior,5,4,15,28.0


In [24]:
priors = priors.join(orders, on='order_id', rsuffix='_')

In [25]:
priors.head()

,order_id,product_id,add_to_cart_order,reordered,order_id_,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,-32416,1,1,2,202279,prior,3,5,9,8.0
1,2,28985,2,1,2,202279,prior,3,5,9,8.0
2,2,9327,3,0,2,202279,prior,3,5,9,8.0
3,2,-19618,4,1,2,202279,prior,3,5,9,8.0
4,2,30035,5,0,2,202279,prior,3,5,9,8.0


In [26]:
priors.drop('order_id_', inplace=True, axis=1)

In [27]:
usr = pd.DataFrame()
# 每个用户平均订单间隔时间 numpy
usr['average_days_between_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)

In [28]:
usr['nb_orders'] = orders.groupby('user_id').size().astype(np.int16)

In [29]:
users = pd.DataFrame()
# 用户购买商品数量
users['total_items'] = priors.groupby('user_id').size().astype(np.int16)

In [30]:
users['all_products'] = priors.groupby('user_id')['product_id'].apply(set)

In [31]:
users['total_distinct_items'] = (users.all_products.map(len)).astype(np.int16)

In [32]:
users.head()

,total_items,all_products,total_distinct_items
user_id,,,
1,59,"{17122, 196, 26405, -19387, 14084, 13032, 2608...",18
2,195,"{-20470, 2573, 18961, 23, -32744, 1559, 22559,...",102
3,88,"{17668, -20853, -17013, 21903, 14992, 21137, 3...",33
4,18,"{21573, -23207, 17769, -30067, -27890, 1200, 1...",17
5,37,"{11777, -24830, 28289, -16761, 20754, 6808, 13...",23


In [33]:
usr.head()

,average_days_between_orders,nb_orders
user_id,,
1,19.000000,11
2,16.285715,15
3,12.000000,13
4,17.000000,6
5,11.500000,5


In [34]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,,
2539329,2539329,1,prior,1,2,8,NaN
2398795,2398795,1,prior,2,3,7,15.0
473747,473747,1,prior,3,3,12,21.0
2254736,2254736,1,prior,4,4,7,29.0
431534,431534,1,prior,5,4,15,28.0


In [35]:
orders.groupby('user_id').size().astype(np.int16).head()

user_id
1    11
2    15
3    13
4     6
5     5
dtype: int16

In [36]:
usr2 = pd.DataFrame()
# 每个用户平均订单间隔时间 numpy
usr2['average_days_between_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)

In [37]:
usr2.head()

,average_days_between_orders
user_id,
1,19.000000
2,16.285715
3,12.000000
4,17.000000
5,11.500000


In [38]:
users = users.join(usr)
del usr

In [39]:
users['average_basket'] = (users.total_items / users.nb_orders).astype(np.float32)
print('user f', users.shape)

user f (206209, 6)


In [40]:
priors['user_product'] = priors.product_id + priors.user_id * 100000

In [41]:
priors.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_product
0,2,-32416,1,1,202279,prior,3,5,9,8.0,-1246968896
1,2,28985,2,1,202279,prior,3,5,9,8.0,-1246907495
2,2,9327,3,0,202279,prior,3,5,9,8.0,-1246927153
3,2,-19618,4,1,202279,prior,3,5,9,8.0,-1246956098
4,2,30035,5,0,202279,prior,3,5,9,8.0,-1246906445


In [42]:
d = dict()
for row in priors.itertuples():
    z = row.user_product
    #组合key不在字典d中初始化
    if z not in d:
        # 1：user_product cnt，
        # 2：获取最近一个订单（订单顺序（最后一个订单），订单id（且订单id最大）），
        # 3：购物车位置累加
        d[z] = (1,
                (row.order_number, row.order_id),
                row.add_to_cart_order)
    else:
        d[z] = (d[z][0] + 1,
                max(d[z][1], (row.order_number, row.order_id)),
                d[z][2] + row.add_to_cart_order)

In [43]:
userXproduct = pd.DataFrame.from_dict(d, orient='index')

In [44]:
userXproduct.head()

,0,1,2
-1246968896,5,"(8, 104690)",10
-1246907495,5,"(6, 132412)",16
-1246927153,1,"(3, 2)",3
-1246956098,5,"(7, 2382766)",24
-1246906445,3,"(7, 2382766)",14


In [45]:
del d
userXproduct.columns = ['nb_orders', 'last_order_id', 'sum_pos_in_cart']

In [46]:
userXproduct.head()

,nb_orders,last_order_id,sum_pos_in_cart
-1246968896,5,"(8, 104690)",10
-1246907495,5,"(6, 132412)",16
-1246927153,1,"(3, 2)",3
-1246956098,5,"(7, 2382766)",24
-1246906445,3,"(7, 2382766)",14


In [47]:
userXproduct.nb_orders = userXproduct.nb_orders.astype(np.int16)
userXproduct.last_order_id = userXproduct.last_order_id.map(lambda x: x[1]).astype(np.int32)
userXproduct.sum_pos_in_cart = userXproduct.sum_pos_in_cart.astype(np.int16)

In [48]:
userXproduct.head()

,nb_orders,last_order_id,sum_pos_in_cart
-1246968896,5,104690,10
-1246907495,5,132412,16
-1246927153,1,2,3
-1246956098,5,2382766,24
-1246906445,3,2382766,14


In [49]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,,
2539329,2539329,1,prior,1,2,8,NaN
2398795,2398795,1,prior,2,3,7,15.0
473747,473747,1,prior,3,3,12,21.0
2254736,2254736,1,prior,4,4,7,29.0
431534,431534,1,prior,5,4,15,28.0


In [50]:
test_orders = orders[orders.eval_set == 'test']

In [51]:
test_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,,
2774568,2774568,3,test,13,5,15,11.0
329954,329954,4,test,6,3,12,30.0
1528013,1528013,6,test,4,3,16,22.0
1376945,1376945,11,test,8,6,11,8.0
1356845,1356845,12,test,6,1,20,30.0


In [52]:
orders2=orders.loc[orders['eval_set']=='test']

In [53]:
orders2.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,,
2774568,2774568,3,test,13,5,15,11.0
329954,329954,4,test,6,3,12,30.0
1528013,1528013,6,test,4,3,16,22.0
1376945,1376945,11,test,8,6,11,8.0
1356845,1356845,12,test,6,1,20,30.0


In [54]:
train_orders = orders[orders.eval_set == 'train']

In [55]:
train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,-16234,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,-15853,4,0
4,1,-21903,5,1


In [56]:
train.set_index(['order_id', 'product_id'], inplace=True, drop=False)

In [66]:
users.head()

,total_items,all_products,total_distinct_items,average_days_between_orders,nb_orders,average_basket
user_id,,,,,,
1,59,"{17122, 196, 26405, -19387, 14084, 13032, 2608...",18,19.000000,11,5.363636
2,195,"{-20470, 2573, 18961, 23, -32744, 1559, 22559,...",102,16.285715,15,13.000000
3,88,"{17668, -20853, -17013, 21903, 14992, 21137, 3...",33,12.000000,13,6.769231
4,18,"{21573, -23207, 17769, -30067, -27890, 1200, 1...",17,17.000000,6,3.000000
5,37,"{11777, -24830, 28289, -16761, 20754, 6808, 13...",23,11.500000,5,7.400000


In [61]:
pd.set_option('display.max_columns', None)

         total_items                                       all_products  \
user_id                                                                   
1                 59  {17122, 196, 26405, -19387, 14084, 13032, 2608...   
2                195  {-20470, 2573, 18961, 23, -32744, 1559, 22559,...   
3                 88  {17668, -20853, -17013, 21903, 14992, 21137, 3...   
4                 18  {21573, -23207, 17769, -30067, -27890, 1200, 1...   
5                 37  {11777, -24830, 28289, -16761, 20754, 6808, 13...   

         total_distinct_items  average_days_between_orders  nb_orders  \
user_id                                                                 
1                          18                    19.000000         11   
2                         102                    16.285715         15   
3                          33                    12.000000         13   
4                          17                    17.000000          6   
5                          23       

In [65]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" 

In [68]:
users.all_products.head()

user_id
1    {17122, 196, 26405, -19387, 14084, 13032, 2608...
2    {-20470, 2573, 18961, 23, -32744, 1559, 22559,...
3    {17668, -20853, -17013, 21903, 14992, 21137, 3...
4    {21573, -23207, 17769, -30067, -27890, 1200, 1...
5    {11777, -24830, 28289, -16761, 20754, 6808, 13...
Name: all_products, dtype: object

In [73]:
priors[priors.user_id==1]

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_product
4089398,431534,196,1,1,1,prior,5,4,15,28.0,100196
4089399,431534,12427,2,1,1,prior,5,4,15,28.0,112427
4089400,431534,10258,3,1,1,prior,5,4,15,28.0,110258
4089401,431534,25133,4,1,1,prior,5,4,15,28.0,125133
4089402,431534,10326,5,0,1,prior,5,4,15,28.0,110326
4089403,431534,17122,6,0,1,prior,5,4,15,28.0,117122
4089404,431534,-23749,7,0,1,prior,5,4,15,28.0,76251
4089405,431534,13176,8,1,1,prior,5,4,15,28.0,113176
4488095,473747,196,1,1,1,prior,3,3,12,21.0,100196
4488096,473747,12427,2,1,1,prior,3,3,12,21.0,112427


In [75]:
train.index

MultiIndex([(      1, -16234),
            (      1,  11109),
            (      1,  10246),
            (      1, -15853),
            (      1, -21903),
            (      1,  13176),
            (      1, -18327),
            (      1,  22035),
            (     36, -25924),
            (     36,  19660),
            ...
            (3421058,  30316),
            (3421058, -29958),
            (3421058,  32650),
            (3421063, -16301),
            (3421063,  13565),
            (3421063,  14233),
            (3421063, -29988),
            (3421070, -29585),
            (3421070,  16953),
            (3421070,   4724)],
           names=['order_id', 'product_id'], length=1384617)

In [76]:
orders.user_id

order_id
2539329         1
2398795         1
473747          1
2254736         1
431534          1
            ...  
2266710    206209
1854736    206209
626363     206209
2977660    206209
272231     206209
Name: user_id, Length: 3421083, dtype: int32

In [77]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,,
2539329,2539329,1,prior,1,2,8,NaN
2398795,2398795,1,prior,2,3,7,15.0
473747,473747,1,prior,3,3,12,21.0
2254736,2254736,1,prior,4,4,7,29.0
431534,431534,1,prior,5,4,15,28.0


In [78]:
def features(selected_orders, labels_given=False):
    print('build candidate list')
    order_list = []
    product_list = []
    labels = []
    i = 0
    for row in selected_orders.itertuples():
        i += 1
        if i % 10000 == 0: print('order row', i)
        order_id = row.order_id
        user_id = row.user_id
        # user_id的不重复商品集合
        user_products = users.all_products[user_id]
        # 产品list，即order_id的候选集
        product_list += user_products
        # 每个product对应当前的order_id,即pair(product_id,order_id)
        order_list += [order_id] * len(user_products)
        # 如果给label，即为train，此时label：如果pair（order_id,product_id）在train中为1，否则为0
        # train中给定的是对应订单，购买的产品。
        if labels_given:
            # 循环判断(order_id, product)是否在train.index中
            labels += [(order_id, product) in train.index for product in user_products]

    df = pd.DataFrame({'order_id': order_list, 'product_id': product_list}, dtype=np.int32)
    labels = np.array(labels, dtype=np.int8)
    del order_list
    del product_list

    print('user related features')
    # map方法：df.order_id每行的值order_id对应orders的索引，且取orders中对应索引的user_id
    df['user_id'] = df.order_id.map(orders.user_id)
    df['user_total_orders'] = df.user_id.map(users.nb_orders)
    df['user_total_items'] = df.user_id.map(users.total_items)
    df['total_distinct_items'] = df.user_id.map(users.total_distinct_items)
    df['user_average_days_between_orders'] = df.user_id.map(users.average_days_between_orders)
    df['user_average_basket'] = df.user_id.map(users.average_basket)

    print('order related features')
    df['order_hour_of_day'] = df.order_id.map(orders.order_hour_of_day)
    df['days_since_prior_order'] = df.order_id.map(orders.days_since_prior_order)
    df['days_since_ratio'] = df.days_since_prior_order / df.user_average_days_between_orders

    print('product related features')
    df['aisle_id'] = df.product_id.map(products.aisle_id)
    df['department_id'] = df.product_id.map(products.department_id)
    df['product_orders'] = df.product_id.map(products.orders).astype(np.int32)
    df['product_reorders'] = df.product_id.map(products.reorders)
    df['product_reorder_rate'] = df.product_id.map(products.reorder_rate)

    print('user_X_product related features')  # 10/13
    df['z'] = df.user_id * 100000 + df.product_id
    df.drop(['user_id'], axis=1, inplace=True)
    # 共同订单数量
    df['UP_orders'] = df.z.map(userXproduct.nb_orders)
    # 共同订单数量在用户所有订单中的占比 p((u,prod_i)|P,u)
    df['UP_orders_ratio'] = (df.UP_orders / df.user_total_orders).astype(np.float32)
    # 共同最后一个订单
    df['UP_last_order_id'] = df.z.map(userXproduct.last_order_id)
    # 物品在该用户订单中的平均位置
    df['UP_average_pos_in_cart'] = (df.z.map(userXproduct.sum_pos_in_cart) / df.UP_orders).astype(np.float32)
    # 最后一次购买这个物品在倒数第几个订单  [1,1,1,0,1,0,1,0,0]
    df['UP_orders_since_last'] = df.user_total_orders - df.UP_last_order_id.map(orders.order_number)
    # 当前订单与最后订单时间差异（hour）
    df['UP_delta_hour_vs_last'] = abs(df.order_hour_of_day - df.UP_last_order_id.map(orders.order_hour_of_day)).map(
        lambda x: min(x, 24 - x)).astype(np.int8)

    df.drop(['UP_last_order_id', 'z'], axis=1, inplace=True)
    print(df.dtypes)
    print(df.memory_usage())
    return df, labels

In [79]:
f_to_use = ['user_total_orders', 'user_total_items', 'total_distinct_items',
            'user_average_days_between_orders', 'user_average_basket',
            'order_hour_of_day', 'days_since_prior_order', 'days_since_ratio',
            'aisle_id', 'department_id', 'product_orders', 'product_reorders',
            'product_reorder_rate', 'UP_orders', 'UP_orders_ratio',
            'UP_average_pos_in_cart', 'UP_orders_since_last',
            'UP_delta_hour_vs_last']

df_train, labels = features(train_orders, labels_given=True)

build candidate list
order row 10000
order row 20000
order row 30000
order row 40000
order row 50000
order row 60000
order row 70000
order row 80000
order row 90000
order row 100000
order row 110000
order row 120000
order row 130000
user related features
order related features
product related features
user_X_product related features
order_id                              int32
product_id                            int32
user_total_orders                     int16
user_total_items                      int16
total_distinct_items                  int16
user_average_days_between_orders    float32
user_average_basket                 float32
order_hour_of_day                      int8
days_since_prior_order              float32
days_since_ratio                    float32
aisle_id                               int8
department_id                          int8
product_orders                        int32
product_reorders                    float32
product_reorder_rate                float32
UP_or

In [80]:
print('Train Columns:', df_train.columns)
df_train[f_to_use].to_csv(IDIR + 'train_feat.csv', index=False)
np.save(IDIR + 'labels.npy', labels)

Train Columns: Index(['order_id', 'product_id', 'user_total_orders', 'user_total_items',
       'total_distinct_items', 'user_average_days_between_orders',
       'user_average_basket', 'order_hour_of_day', 'days_since_prior_order',
       'days_since_ratio', 'aisle_id', 'department_id', 'product_orders',
       'product_reorders', 'product_reorder_rate', 'UP_orders',
       'UP_orders_ratio', 'UP_average_pos_in_cart', 'UP_orders_since_last',
       'UP_delta_hour_vs_last'],
      dtype='object')


In [81]:
import xgboost as xgb
import pandas as pd
import numpy as np
import lightgbm as lgb
from datetime import datetime
from sklearn.model_selection import train_test_split

In [82]:
IDIR = 'G://bigdata//badou//00-data//'
df_train = pd.read_csv(IDIR + 'train_feat.csv').fillna(0.).to_sparse()
labels = np.load(IDIR + 'labels.npy')

X_train, X_test, y_train, y_test = train_test_split(df_train, labels,
                                                    test_size=0.2,
                                                    random_state=2020)

d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: DataFrame.to_sparse is deprecated and will be removed in a future version
  
d:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3606: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  result = self._constructor(new_data).__finalize__(self)


In [83]:
df_train.head()

d:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3456: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  return klass(values, index=self.index, name=items, fastpath=True)
d:\ProgramData\Anaconda3\lib\site-packages\pandas\core\sparse\frame.py:745: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  default_fill_value=self._default_fill_value,
d:\ProgramData\Anaconda3\lib\site-packages\pandas\core\sparse\frame.py:721: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See

,user_total_orders,user_total_items,total_distinct_items,user_average_days_between_orders,user_average_basket,order_hour_of_day,days_since_prior_order,days_since_ratio,aisle_id,department_id,product_orders,product_reorders,product_reorder_rate,UP_orders,UP_orders_ratio,UP_average_pos_in_cart,UP_orders_since_last,UP_delta_hour_vs_last
0,11,59,18,19.0,5.363637,8,14.0,0.736842,24,4,13880,9377.0,0.675576,1,0.090909,6.0,6,7
1,11,59,18,19.0,5.363637,8,14.0,0.736842,77,7,35791,27791.0,0.776480,10,0.909091,1.4,1,0
2,11,59,18,19.0,5.363637,8,14.0,0.736842,54,17,1214,536.0,0.441516,2,0.181818,5.0,7,1
3,11,59,18,19.0,5.363637,8,14.0,0.736842,77,7,8558,6953.0,0.812456,3,0.272727,3.0,1,0
4,11,59,18,19.0,5.363637,8,14.0,0.736842,91,16,15935,12923.0,0.810982,1,0.090909,2.0,10,0


In [85]:
print(type(df_train))

<class 'pandas.core.sparse.frame.SparseDataFrame'>


In [86]:
# ########################### XGB ##################

dtrain = xgb.DMatrix(X_train, y_train)
dval = xgb.DMatrix(X_test, y_test)

In [87]:
param = {'booster': 'gbtree',
         'gamma': 0.1,
         'subsample': 0.8,
         'colsample_bytree': 0.8,
         'max_depth': 6,
         'eta': 0.03,
         'silent': 1,
         'objective': 'binary:logistic',
         'nthread': 4,
         'eval_metric': 'auc'}
watchlist = [(dtrain, 'train'), (dval, 'val')]

In [88]:
xgb_start = datetime.now()
model = xgb.train(param, dtrain, num_boost_round=100, evals=watchlist)
xgb_stop = datetime.now()
execution_time_lgbm = xgb_stop - xgb_start
print('xgb time cost: ', execution_time_lgbm / 1000)
# 对测试集进行预测
dtest = xgb.DMatrix(X_test)
ans = model.predict(dtest)
#
# 计算准确率
cnt1 = 0.
cnt2 = 0.
for i in range(len(y_test)):
    if ans[i] > 0.3:
        cnt1 += 1.
    else:
        cnt2 += 1.

print("Accuracy: %.2f %% " % (100 * cnt1 / (cnt1 + cnt2)))

[09:51:00] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:51:00] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\gbm\gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	train-auc:0.81944	val-auc:0.81873
[1]	train-auc:0.82044	val-auc:0.81983
[2]	train-auc:0.82261	val-auc:0.82205
[3]	train-auc:0.82343	val-auc:0.82291
[4]	train-auc:0.82359	val-auc:0.82308
[5]	train-auc:0.82401	val-auc:0.82350
[6]	train-auc:0.82433	val-auc:0.82381
[7]	train-auc:0.82442	val-auc:0.82391
[8]	train-auc:0.82439	val-auc:0.82387
[9]	tra

In [89]:
# ##################lightGBM#########################
params_lgbm = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'binary',
    # 'objective': 'regression', # 目标函数     ####regression默认regression_l2
    # 'metric': 'rmse',  # 评估函数
    'metric': 'auc',
    'max_depth': 6,  #   树的深度           ###按层
    'num_leaves': 50,  #   由于leaves_wise生长，小于2^max_depth   #####按leaf_wise
    'learning_rate': 0.05,  # 学习速率,步长
    'subsample': 0.8,  #  数据采样
    'colsample_bytree': 0.8,  #  特征采样
}

train_data = lgb.Dataset(X_train, y_train)
val_data = lgb.Dataset(X_test, y_test)

num_round = 100
start = datetime.now()
lgbm = lgb.train(params_lgbm, train_data,
                 num_round,
                 valid_sets=[train_data, val_data],
                 early_stopping_rounds=50)
stop = datetime.now()
# lightgbm
execution_time_lgbm = stop - start
print('lightgbm time cost: ', execution_time_lgbm)

d:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5997: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  return self._constructor(data).__finalize__(self)
d:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3606: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  result = self._constructor(new_data).__finalize__(self)


[1]	training's auc: 0.816678	valid_1's auc: 0.816151
Training until validation scores don't improve for 50 rounds
[2]	training's auc: 0.818323	valid_1's auc: 0.81776
[3]	training's auc: 0.82242	valid_1's auc: 0.821809
[4]	training's auc: 0.822711	valid_1's auc: 0.822133
[5]	training's auc: 0.823847	valid_1's auc: 0.823278
[6]	training's auc: 0.824624	valid_1's auc: 0.824064
[7]	training's auc: 0.824795	valid_1's auc: 0.824265
[8]	training's auc: 0.824808	valid_1's auc: 0.824292
[9]	training's auc: 0.825415	valid_1's auc: 0.824871
[10]	training's auc: 0.825544	valid_1's auc: 0.82499
[11]	training's auc: 0.825606	valid_1's auc: 0.825054
[12]	training's auc: 0.825614	valid_1's auc: 0.825064
[13]	training's auc: 0.825622	valid_1's auc: 0.825065
[14]	training's auc: 0.826089	valid_1's auc: 0.825528
[15]	training's auc: 0.826101	valid_1's auc: 0.825526
[16]	training's auc: 0.826189	valid_1's auc: 0.825612
[17]	training's auc: 0.826408	valid_1's auc: 0.825836
[18]	training's auc: 0.826536	val

In [90]:
ans

array([0.03615328, 0.22884874, 0.05484031, ..., 0.04004969, 0.04119232,
       0.04228274], dtype=float32)

In [91]:
model